In [13]:
import re
import json
from post_process import process
# from seq2seq.metrics.lc_quad_2.post_process import process

In [26]:
sparql = data[44]['prediction']

sparql2 = data[44]['label']

In [28]:
p1 = re.compile(r'[{](.*?)[}]', re.S)  #最小匹配
    # p2 = re.compile(r'[{](.*)[}]', re.S)   #贪婪匹配
result1 = re.findall(p1, process(sparql))
result2 = re.findall(p1, process(sparql2))

In [29]:
result2

['wd:Space Invaders wdt:publisher ?vr0 . ?vr0 wdt:instance of wd:business']

In [40]:
query_match_one(process(sparql), process(sparql2))

{'?vr0wdt:publisherwd:spaceinvaders', '?vr0wdt:instanceofwd:business'}
{'?vr0wdt:instanceofwd:business', 'wd:spaceinvaderswdt:publisher?vr0'}


0

In [43]:
def parse_triples(sparql):
    p1 = re.compile(r'[{](.*?)[}]', re.S)  #最小匹配
    # p2 = re.compile(r'[{](.*)[}]', re.S)   #贪婪匹配
    result1 = re.findall(p1, sparql)
    # result2 = re.findall(p2, sparql)
    # print(sparql)
    if len(result1)>0:
        triple_l = result1[0].strip().split(". ")
        triple_l = [triple.strip().replace(" .", "").strip().replace(" ", "").lower() for triple in triple_l]
        triple_s = set(triple_l)
    else:
        triple_s = set()
    return triple_s


def query_match_one(prediction, reference):
    pred_s = parse_triples(prediction)
    ref_s = parse_triples(reference)
    
    if pred_s == ref_s:
        return 1
    else:
        return 0


def compute_query_match_metric(data):
    qm, total = 0, 0
    for item in data:
        qm += query_match_one(process(item['prediction']), process(item["label"]))
        # qm += query_match_one(item['prediction'], item["label"])
        total += 1
        
    return {
        "query_match": float(qm/total),
    }

In [44]:
compute_query_match_metric(data)

{'query_match': 0.9123387363546146}

In [25]:
for i,item in enumerate(data):
    if process(item['prediction']) != process(item["label"]):
        print(i)
        print(process(item['prediction']))
        print(process(item["label"]))

44
select distinct ?vr0 where {?vr0 wdt:publisher wd:Space Invaders . ?vr0 wdt:instance of wd:business}
select distinct ?vr0 where {wd:Space Invaders wdt:publisher ?vr0 . ?vr0 wdt:instance of wd:business}
47
select distinct ?vr0 where {wd:intelligence wdt:measures ?vr0 . ?vr0 wdt:instance of wd:IQ test}
select distinct ?vr0 where {?vr0 wdt:measures wd:intelligence . ?vr0 wdt:instance of wd:IQ test}
50
ask where {wd:Brazilian Military Cemetery of Pistoia wdt:quantity buried ?vr0 filter (?vr0 > 369.6)}
ask where {wd:Brazilian Military Cemetery of Pistoia wdt:quantity buried ?vr0 filter (?vr0=369.6)}
78
select distinct ?vr0 where {wd:Prince George of Cambridge wdt:child ?vr0 . ?vr0 wdt:instance of wd:human}
select distinct ?vr0 where {?vr0 wdt:child wd:Prince George of Cambridge . ?vr0 wdt:instance of wd:human}
81
select distinct ?vr0 where {?vr0 wdt:statement describes wd:Pythagorean theorem}
select distinct ?vr0 where {wd:Pythagorean theorem wdt:statement describes ?vr0}
101
select (cou

In [6]:
path = '../../../train/train_11.02/predictions_eval_657.4232804232804.json'
data = json.load(open(path, 'rb'))